In [1]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from requests_html import HTML

In [ ]:
def filter_by_response(response) -> bool:
    return ('\"incomplete_results\":true' not in response.text)
requests_cache.install_cache(expire_after=None, filter_fn=filter_by_response)

In [ ]:
url = "https://soundcloud.com/discover/sets/charts-top:all-music:vn"
outputFile = "/Users/lanmac/Study/output.tsv"

In [ ]:
browser = webdriver.Chrome()
browser.get(url)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [ ]:
html = browser.page_source

In [ ]:
soup = BeautifulSoup(html)

In [ ]:
posts = soup.find_all("li", "systemPlaylistTrackList__item")

for post in posts:
    user = post.find("a", "trackItem__username").get_text()
    print(user)
    
    track = post.find("a", "trackItem__trackTitle").get_text()
    print(track)
print(len(posts))

________________________________________

**Lấy tất cả tracks của một user**

In [101]:
users = []
tracks = []

In [53]:
url = "https://soundcloud.com/discover"

In [54]:
browser = webdriver.Chrome()
browser.get(url)

In [55]:
html = browser.page_source
soup = BeautifulSoup(html)

In [56]:
# Đồng ý cookies
cookies = browser.find_element(By.XPATH,"//button[@id='onetrust-accept-btn-handler']")
cookies.click()

# Chọn một tab genre (demo)
time.sleep(2)
genre = browser.find_element(By.XPATH,"//a[@class='playableTile__heading playableTile__mainHeading sc-truncate sc-type-light sc-text-secondary sc-font-light sc-link-dark sc-link-primary sc-text-h4']")
genre.click()

In [57]:
def scroll_down_till_end():
    # Loop 2 lần cho chắc ăn
    for i in range(2):
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        lastCount = 0
        match=False
        while(match==False):
            time.sleep(5)
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount==lenOfPage:
                match=True
            else: 
                lastCount = lenOfPage

In [67]:
def collect_tracks_playlist():
    time.sleep(2)
    html = browser.page_source
    soup = BeautifulSoup(html)
    
    posts = soup.find_all("div", "trackItem__content")

    for post in posts:
        user = post.find("a", "trackItem__username").get_text()
        if user not in users:
            users.append(user)

        track = post.find("a", "trackItem__trackTitle").get_text()
        tracks.append(track)

In [68]:
scroll_down_till_end()
collect_tracks()

In [69]:
tracks

['Tell Ur Mom II - Winno (ft Heily)',
 'PHI HÀNH GIA - RENJA x SLOW T x LIL WUYN x KAIN x SUGAR CANE',
 'Lạc - Rhymastic',
 'Nếu mai chia tay ( Hieu Tran Chill RnB Version) - Monstar Ft. Amee',
 'LeeHi (이하이) - ONLY',
 'không phải tại nó // lạc - mai ngô ft rhymastic',
 'Biết em - Lynk Nguyễn',
 '[DCOD] NGTANOISE (Người ta nói - remake) | VSOUL x MFREE x TUYEN VO|',
 'GPS - NIDEBIG ft BINQANG , PURR ( PROD : HS YAFU )',
 'Obito Ft. Hnhngan Live YOUR SMILE - Emma X Seachains X Obito',
 '3107-3 (lofi ver. by VUX) - W/n x Nâu x Duongg x Titie',
 'Tell Ur Mom II (Remix)Winno ft Heily',
 'Khi co don em nho ai',
 'HIEUTHUHAI - NGHE NHƯ TÌNH YÊU (FELIKS ALVIN REMIX) | HAWYS EDIT',
 'DAU CAN MOT BAI CA TINH YEU',
 'Việt - Hơn Cả Mây Trời (lofi ver by hawys)',
 'Nếu Anh Thấy Lòng Mình Yếu Đuối - Tùa x CM1X',
 'Tlinh - Thích Quá Rùi Nà ft. Trung Trần (prod by @rpt.phongkhin )',
 'SOOBIN - THÁNG NĂM (SPECIAL VERSION) FT. SLIM V',
 'Orange x Hoàng Dũng - Khi Em Lớn (Lo-fi ver by Hawys)',
 'THANH XU

In [81]:
time.sleep(2)
profile = browser.find_element(By.XPATH,"//a[@class='trackItem__username sc-link-light sc-link-secondary']")
profile.click()

In [98]:
def get_all_tracks():
    #time.sleep(3)
    scroll_down_till_end()
            
    time.sleep(3)
    html = browser.page_source
    soup = BeautifulSoup(html)
    
    time.sleep(3)
    songs = soup.find_all("li", "soundList__item")
    for track in songs:
        title = track.find("a", "soundTitle__title").get_text().strip()
        if title not in tracks:
            tracks.append(title)

In [99]:
get_all_tracks()

In [100]:
tracks

['Tell Ur Mom II - Winno (ft Heily)',
 'PHI HÀNH GIA - RENJA x SLOW T x LIL WUYN x KAIN x SUGAR CANE',
 'Lạc - Rhymastic',
 'Nếu mai chia tay ( Hieu Tran Chill RnB Version) - Monstar Ft. Amee',
 'LeeHi (이하이) - ONLY',
 'không phải tại nó // lạc - mai ngô ft rhymastic',
 'Biết em - Lynk Nguyễn',
 '[DCOD] NGTANOISE (Người ta nói - remake) | VSOUL x MFREE x TUYEN VO|',
 'GPS - NIDEBIG ft BINQANG , PURR ( PROD : HS YAFU )',
 'Obito Ft. Hnhngan Live YOUR SMILE - Emma X Seachains X Obito',
 '3107-3 (lofi ver. by VUX) - W/n x Nâu x Duongg x Titie',
 'Tell Ur Mom II (Remix)Winno ft Heily',
 'Khi co don em nho ai',
 'HIEUTHUHAI - NGHE NHƯ TÌNH YÊU (FELIKS ALVIN REMIX) | HAWYS EDIT',
 'DAU CAN MOT BAI CA TINH YEU',
 'Việt - Hơn Cả Mây Trời (lofi ver by hawys)',
 'Nếu Anh Thấy Lòng Mình Yếu Đuối - Tùa x CM1X',
 'Tlinh - Thích Quá Rùi Nà ft. Trung Trần (prod by @rpt.phongkhin )',
 'SOOBIN - THÁNG NĂM (SPECIAL VERSION) FT. SLIM V',
 'Orange x Hoàng Dũng - Khi Em Lớn (Lo-fi ver by Hawys)',
 'THANH XU